In [1]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os
import json

# 3rd party lib
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook
#matplotlib.rcParams.update({'font.size': 22})
#from bqplot import pyplot as plt

import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt
from gphelper import GPHelper

from importlib import reload

import warnings
warnings.filterwarnings('ignore')


from multiprocessing import Pool

Using matplotlib backend: Qt5Agg


In [3]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    if img.min() <= 0:
        img += epsilon
    return img

def get_spheroids(num=10, band='h'):
    with open('../spheroids', 'r') as f:
        spheroids= np.array(f.readlines())
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)
    
    sources = []
    for s in spheroids:
        s = s.strip()
        img = fits.getdata(f_string.format(s, band))
        segmap = fits.getdata(f_string.format(s, 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img
        del segmap
        
    return sources
        
def process_img(args):
    def denoise(img, segmap, img_id):
        noise_bank = img[segmap==0].flatten()
        other_source = np.logical_and(segmap!=0, segmap!=img_id)
        np.place(img, other_source, noise_bank)

        sep_arr = img.byteswap().newbyteorder('N')
        bkg = sep.Background(sep_arr, mask=segmap==img_id, bw=10, bh=10)
        img = img-bkg

        return img
    
    def get_rs_and_fs(img, src_map):
        cx, cy = it.img_center(img, src_map)
        xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
        rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))

        rs = rs.flatten()
        fs = img.flatten()
        sorted_rs = np.argsort(rs)

        rs = rs[sorted_rs]
        fs = fs[sorted_rs]

        return rs, fs
    
    img, segmap, img_id = args
    src_map = segmap==img_id
    img = denoise(img, segmap, img_id)
    
    rs, fs = get_rs_and_fs(img, src_map)
    
    dr = 0.05*(rs.max()-rs.min())
    fs = dt.loessc_p(rs, fs, dr, pnum=2)
    fs[fs<0]=0
    
    num_interp = 100
    _rs = np.linspace(rs.min(), rs.max(), num_interp)
    fs = np.interp(_rs, rs, fs)
    rs = _rs
    
    L = np.zeros(num_interp)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,num_interp,1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*((fs[i]+fs[i-1])/2)

    A = np.pi * rs**2
    etas = fs*A/L
    r_lim = rs<25.0
    #etas = etas[r_lim]
    mono_dec = np.argmax(np.diff(etas)>=0)
    mono_dec = np.arange(len(etas))<mono_dec
    r_lim = np.logical_and(r_lim, mono_dec)
    
    if np.all(r_lim==False):
        #plt.plot(rs, fs)
        plt.figure()
        plt.plot(rs[:-1],np.diff(etas))
        plt.figure()
        plt.plot(rs, etas)
        #plt.figure()
        #plt.plot(np.flipud(etas), np.flipud(rs))
        #plt.figure()
        #plt.title(img_id)
        #plt.imshow(img)
    
    Rp = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    Fp_idx = np.square(rs - 2*Rp).argmin()

    Fp = np.cumsum(L[:Fp_idx]/L[:Fp_idx].sum())
    Ip = np.interp(Rp, rs, fs)

    return (rs, fs, Rp, Ip, etas, img_id)

In [4]:
# get imgs in other bands
spheroids_j = get_spheroids(-1, 'j')
spheroids_v = get_spheroids(-1, 'v')
spheroids_z = get_spheroids(-1, 'z')

In [5]:
# get the imgs in h band 
with open('h_vals.json', 'r') as f:
    processed_h = json.load(f)

In [9]:
ids = [h['id'] for h in processed_h]

# get rid of the imgs that were excluded from h
tmpj, tmpv, tmpz = [], [], []
for j, v, z, in zip(spheroids_j, spheroids_v, spheroids_z):
    if j[-1] in ids:
        tmpj.append(j)
        tmpv.append(v)
        tmpz.append(z)

spheroids_j = tmpj
spheroids_v = tmpv
spheroids_z = tmpz

496 496 496
